In [73]:
import pandas as pd

In [75]:
ascend_active_attendance = pd.read_csv('data/ascend_active_attendance.csv')
ascend_former_attendance = pd.read_csv('data/ascend_former_attendance.csv')

In [77]:
def cleanAttendance(attendance_file):
    
    # Drop empty rows
    attendance_file.dropna(subset=['Student.1'], inplace = True)

    # Chose subset of columns to keep
    attendance_file = attendance_file[['Student', 'Grade', 'Homeroom', 'Member', 'Present', 'Absent', 'Tardy', 'Dismiss']]

    # filter out renments of excel pagination
    attendance_file = attendance_file.loc[attendance_file['Student'] != "CHARTER"]
    attendance_file = attendance_file.loc[attendance_file['Student'] != "Student"]
    attendance_file = attendance_file.loc[~attendance_file['Student'].str.contains("Page")]
    
    return(attendance_file)

In [78]:
ascend_active_attendance_clean = cleanAttendance(ascend_active_attendance)
ascend_former_attendance_clean = cleanAttendance(ascend_former_attendance)

In [81]:
ascend_active_attendance_clean.to_csv('data/ascend_active_attendance_final.csv')

In [82]:
ascend_former_attendance_clean.to_csv('data/ascend_former_attendance_final.csv')